# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [8]:
from pymongo import MongoClient
import pandas as pd
import time

In [9]:
client = MongoClient("localhost:27017")

In [10]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [11]:
client.list_database_names()

['admin', 'config', 'local']

In [12]:
db = client["Ironhack"]

In [13]:
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [17]:
list(c.find({"name":"Babelgum"}, projection = {"_id":0, "name":1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [18]:
condition_1 = {"number_of_employees": {"$gt": 5000}}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find(condition_1, projection).sort("number_of_employees", -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [33]:
condition_1 = {"founded_year": {"$gte": 2000, "$lte": 2005 }}
projection = {"_id":0, "name":1, "founded_year":1}
list(c.find(condition_1, projection).sort("founded_year",1).limit(5))

[{'name': 'MeeVee', 'founded_year': 2000},
 {'name': 'Steorn', 'founded_year': 2000},
 {'name': 'Boonex', 'founded_year': 2000},
 {'name': 'PhotoBox', 'founded_year': 2000},
 {'name': 'AllofMP3', 'founded_year': 2000}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [55]:
list(c.find({"$and": 
             [{"ipo.valuation_amount": {"$gt": 100000000}}, 
                  {"founded_year": {"$lte": 2010 }}]}, 
            {"_id": 0, "name":1, "ipo":1 }).sort("ipo", 1).limit(3))

[{'name': 'Salesforce',
  'ipo': {'valuation_amount': 110000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2004,
   'pub_month': 7,
   'pub_day': 2,
   'stock_symbol': 'NYSE:CRM'}},
 {'name': 'Geeknet',
  'ipo': {'valuation_amount': 134000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2010,
   'pub_month': 11,
   'pub_day': 2,
   'stock_symbol': 'GKNT'}},
 {'name': 'QuinStreet',
  'ipo': {'valuation_amount': 140000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2010,
   'pub_month': 1,
   'pub_day': 11,
   'stock_symbol': 'NASDAQ:QNST'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [38]:
condition_1 = {"number_of_employees": {"$lt": 5000}}
condition_2 = {"founded_year": {"$lte": 2005 }}
projection = {"_id":0, "name":1, "founded_year":1, "number_of_employees":1}

list(c.find({"$and": [condition_1, condition_2]}, projection).sort("number_of_employees", 1).limit(10))

[{'name': 'Skype', 'number_of_employees': 0, 'founded_year': 2003},
 {'name': 'Ticketmaster', 'number_of_employees': 0, 'founded_year': 1976},
 {'name': 'MindTouch', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Fox Interactive Media',
  'number_of_employees': 0,
  'founded_year': 1979},
 {'name': 'Bebo', 'number_of_employees': 0, 'founded_year': 2005},
 {'name': 'YouTube', 'number_of_employees': 0, 'founded_year': 2005},
 {'name': 'Attributor', 'number_of_employees': 0, 'founded_year': 2005},
 {'name': 'Yeigo', 'number_of_employees': 0, 'founded_year': 2005},
 {'name': 'PartnerUp', 'number_of_employees': 0, 'founded_year': 2005},
 {'name': 'stylediary', 'number_of_employees': 0, 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [50]:
projection = {"_id":0, "name":1}
list(c.find({"partners" : {"$exists": "False"}}, projection).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [60]:
projection = {"_id":0, "name":1, "category_code":1}
list(c.find({"category_code": None}, projection).limit(3))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [48]:
condition_1 = {"number_of_employees": {"$gt": 100, "$lt": 1000 }}
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find(condition_1, projection).sort("number_of_employees",1).limit(5))

[{'name': 'KAYAK', 'number_of_employees': 101},
 {'name': 'Jedox', 'number_of_employees': 101},
 {'name': 'OfficialVirtualDJ', 'number_of_employees': 102},
 {'name': 'OSA Technologies', 'number_of_employees': 102},
 {'name': 'GoGrid', 'number_of_employees': 102}]

### 9. Order all the companies by their IPO price in a descending order.

In [53]:
list(c.find(projection = {"_id":0, "name":1, "ipo":1}).sort("ipo",-1).limit(3))

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000.0,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [66]:
projection = {"_id":0, "name":1, "number_of_employees":1}
list(c.find({}, projection).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [71]:
condition_1 = {"founded_month": {"$gte": 6, "$lte": 12 }}
projection = {"_id":0, "name":1, "founded_month":1}
list(c.find(condition_1, projection).limit(5))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [91]:
condition_1 = {"founded_year": {"$lte": 2000 }}
condition_2 = {"acquisition.price_amount": {"$gt": 10000000}}
projection = {"_id":0, "name":1, "founded_year":1, "acquisition.price_amount":1}

list(c.find({"$and": [condition_1, condition_2]}, projection).sort("number_of_employees", 1).limit(5))

[{'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Sabre',
  'founded_year': 1960,
  'acquisition': {'price_amount': 4300000000.0}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [76]:
condition_1 = {'acquisition.acquired_year': {"$gt": 2010}}
projection = {"_id":0, "name":1, "acquisition":1}
list(c.find(condition_1, projection).sort("acquisition.price_amount", -1).limit(3))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [81]:
condition_1 = {'founded_year': {"$ne": None}}
projection = {"_id":0, "name":1, "founded_year":1}
list(c.find(condition_1, projection).sort("founded_year", 1).limit(5))

[{'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [85]:
condition_1 = {"founded_day": {"$gte": 1, "$lte":7 }}
projection = {"_id":0, "name":1, "founded_day":1}
list(c.find(condition_1, projection).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'BlueLithium', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [89]:
condition_1 = {"category_code": "web"}
condition_2 = {"number_of_employees": {"$gt": 4000}}
projection = {"_id":0, "name":1, "number_of_employees":1, "category_code":1}

list(c.find({"$and": [condition_1, condition_2]}, projection).sort("number_of_employees", 1).limit(5))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [93]:
condition_1 = {"acquisition.price_amount": {"$gt": 10000000}}
condition_2 = {"acquisition.price_currency_code": "EUR"}
projection = {"_id":0, "name":1, "acquisition.price_amount":1, "acquisition.price_currency_code":1}

list(c.find({"$and": [condition_1, condition_2]}, projection).limit(5))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [94]:
condition_1 = {"founded_month": {"$gte": 1, "$lte": 3 }}
projection = {"_id":0, "name":1, "founded_month":1}
list(c.find(condition_1, projection).limit(5))

[{'name': 'Facebook', 'founded_month': 2},
 {'name': 'Twitter', 'founded_month': 3},
 {'name': 'StumbleUpon', 'founded_month': 2},
 {'name': 'Gizmoz', 'founded_month': 1},
 {'name': 'Helio', 'founded_month': 2}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [95]:
condition_1 = {"founded_year": {"$gte": 2000, "$lte":2010 }}
condition_2 = {"acquisition.acquired_year":{"$gt": 2011}}
projection = {"_id":0, "name":1, "founded_year":1, "acquisition.acquired_year":1}

list(c.find({"$and": [condition_1, condition_2]}, projection).limit(5))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [96]:
# The database has some deadpooled years with the actual year (e.g: 2018) 
#and some others deadpooled years with the number of years (e.g: 3).
#Unsure of how to work with this difference. Thought about $substract deadpooled_year - founded_year to get the difference
#but not sure how to do that with the code. 

deadpooled_year = founded_year - deadpooled_year
condition_1 = {"deadpooled_year": {"$gt": 3}}
projection = {"_id":0, "name":1, "deadpooled_year":1}
list(c.find(condition_1, projection).limit(5))

[{'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Sparter', 'deadpooled_year': 2008},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008}]

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```